In [1]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from huggingface_hub import login
from dotenv import load_dotenv
import os
from langchain_community.vectorstores import FAISS

load_dotenv()
HF_API_TOKEN = os.environ["HF_API_TOKEN"]

c:\Users\oneas\ONEASH_local\LLM-KTAS\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task = 'feature-extraction',
    huggingfacehub_api_token=HF_API_TOKEN
)

In [3]:
db = FAISS.load_local('db/KTAS_adult_faiss', hf_embeddings, allow_dangerous_deserialization=True)

### 테스트 1: LLM으로 바로 주증상 선택

In [23]:
system_prompt = """You are a specialized medical symptom classification assistant. You will receive a patient’s clinical notes and you must choose exactly one symptom from a predefined list of <possible symptoms>. When you provide your answer, you must not add any additional words, symbols, or reasoning in your final response.

<possible symptoms>
20주 미만의 임신
20주 이상의 임신
감각상실 / 이상감각
감염 가능성 확인
감염성 질환에 노출
객혈
경증 및 비특이적 증상 호소
고혈당
고혈압
고환 통증 / 부종
과다 복용
과다호흡증후군
관절 부종
구토 / 구역
국소성 부종 / 발적
귀의 삼출물
귀의 손상
귀의 이물질
기괴한 행동
기억상실
기침 / 코막힘
기타 피부 상태
눈 충혈 / 분비물
눈 통증
눈부심
눈의 외상
눈의 이물질
눈의 화학물질 노출
다뇨증
다리 부기 / 부종
단독 복부 외상 - 관통상
단독 복부 외상 - 둔상
단독 흉부 외상 - 관통상
단독 흉부 외상 - 둔상
동상 / 한랭손상
두부 손상
두통
드레싱 교체
딸꾹질
떨림
맥박이 없거나 차가운 사지
목, 등, 허리 통증
목의 부종 / 통증
목의 외상
몸통을 포함한 다발성 외상 - 관통상
몸통을 포함한 다발성 외상 - 둔상
물림
물질 금단
물질 오용 / 중독
반지 제거
발작
발진
변비
보행 장애 / 운동 실조 / 강직
복뷔 종괴 / 팽만
복시
복통
불면증
불안 / 위기 상황
비정상 검사 결과
사지 약화 / 뇌졸중 증상
사회문제
상기도감염 증상 호소
상지 손상
상지 통증
상처 확인
샅고랑 부위 통증  종괴
생식기의 분비물 / 피부병변
생식기의 외상
석고 붕대 확인
설사
성폭행
소변배출장애
소양증
수술 후 합병증
숨참
스테이플 / 봉합사 제거
시력장애
식욕부진
실신 / 전실신
심계항진 / 불규칙한 심장박동
심정지(비외상성)
심정지(외상성)
쏘임
안면 통증 (비외상성 / 비치아성)
안명 외상
안와주위 부종
알레르기 반응
연하장애 / 연하곤란
열
열상 / 천공
영상 검사 / 검사실 검사
옆구리 통증
온열손상
외상성 목, 등, 허리 손상
외상없이 저절로 멍듦
요로감염 증상
우울증 / 자살 / 자해
월경 문제
유방의 발적 / 압통
유해물질 흡입
음경 부종
음순 부종
의료 장비 문제
의식 수준의 변화
이명
이물질 삼킴
이통
익수
인후통
일측성의 홍조 띤 뜨거운 사지
저체온증
저혈당
전기 손상
전문 진료를 위해서 의뢰된 환자
전신 부종
전신 쇠약
절단
직장내 이물질
질 분비물
질 출혈
질 통증 / 가려움
질내 이물질 
착란
찰과상
창백함 / 빈혈
처방전 / 투약 문의
청력소실
청색증
출산 후 문제 (6주 이내)
치아 / 구강 문제
코막힘
코의 외상
코의 이물질
코피
토혈
폭력 / 살인 행위
피부의 이물질
핍뇨증
하지 손상 
하지 통증
항문 / 직장 / 회음부 외상
항문 / 직장 / 회음부 통증
현훈
혈뇨
혈변 / 흑색변
혈액이나 체액에 노출
호흡 정지
호흡기 이물질
혹, 돌기, 굳은 살
화상
화학물질 노출
환각 / 망상
환자의 안녕에 대한 고려 (학대, 방임)
황달
흉통 (비심장성)
흉통 (심장성)"""

In [ ]:
user_prompt = """Read the following information carefully:

<Chief Complaint>: 
{주호소}

<Additional Notes>: 
{비고}

Analyze the <Chief Complaint> and <Additional Notes> to determine which single symptom from the <possible symptom> best matches the information given.

Important Constraints:
- You must choose only one symptom from the list.
- Your answer must exactly match the corresponding symptom in the <possible symptoms> (including spacing, punctuation, letter case, etc.).
- Do not output any extra words, sentences, or explanations. Output only the final answer."""